In [ ]:
#Sprint 5
import openai
import typing
from typing import List, Tuple

import tiktoken
import ast


import pandas as pd
import time

catName = 'Category'
respName = 'Response'

In [ ]:
def SetKeyWithFile(keyFile = '/Users/adi/Desktop/RandomCode/Confidential/ChatGPTApiKey.txt') -> str:
    #Opens the File with the API in it
    text = open(keyFile, 'r')
    myKey = text.read()
    text.close()
    #Sets API key to file location
    openai.api_key = myKey
    return myKey

def exceltoDF(sampleData = "/Users/adi/Desktop/RandomCode/StudentSurveyNaturalLanguageProcessing/Book1.xlsx") -> object:
    df = pd.read_excel(sampleData)
    return df

SetKeyWithFile()
df = exceltoDF()
df

def putinDict(df: object) -> dict:
    #Puts it all into a dictionary according to type
    responses_by_category = {}
    for category in df[catName ].unique():
        responses_by_category[category] = df[df[catName ] == category][respName].tolist()
    return responses_by_category
dic = putinDict(df)

def count_token(string):
    encoding = tiktoken.encoding_for_model('text-davinci-003')
    return len(encoding.encode(string))

def generate_augmented_examples(prompt, times):
    retries = 3
    n = 10
    token_count = count_token(prompt)
    hol = []
    count = 0
    errors = 0
    while (count < times):
        try:
            # Use OpenAI API to generate completions based on the prompt
            print("try #" + str(count + 1))
            if(count == 0):
                prompt = "Give me even more data regarding the following." + prompt
                token_count = count_token(prompt)
            response = openai.Completion.create(
                #NOTE: The orignal version has been depreceated
                #gpt-3.5-turbo-instruct should replace it
                #It was orignally: 
                #engine="text-davinci-003",
                engine="gpt-3.5-turbo-instruct",
                prompt=prompt,
                max_tokens=4000 - token_count,
                n = n   # You can adjust this parameter based on your needs
            )
            print("try number " + (count+1) + " done")

            first = [choice['text'].strip() for choice in response['choices']]
            hol.append(first)
            count = count + 1
        except openai.error.RateLimitError as e:
            print(e)
            errors = errors + 1
            if(errors == 10):
                return flatten(hol)
            else:
                # Sleep for a moment and retry
                time.sleep(22)  # Adjust the sleep duration as needed
                count = count - 1
                continue
            return flatten(hol)
            
    
    return flatten(hol)
    print("Its GG we out of retries")

#Flattens a list of various responses into a one dimension list
def flatten(lst, split = '\n'):
    result = []
    for item in lst:
        if isinstance(item, list):
            result.extend(flatten(item))
        elif isinstance(item, str):
            result.extend(item.split(split))
        else:
            result.append(item)
    return result
#Turns a list into a data frame
def listDf(lst, column_name = 'name'):
    flattened = flatten(lst)

    columns = ['name']
    time = pd.DataFrame(flattened, columns=columns)
#Saves a list into a txt file
def list_to_txt(file_name, my_list):
    try:
        with open(file_name, 'w') as file:
            file.write(str(my_list))
        print(f"List successfully written to {file_name}")
    except Exception as e:
        print(f"Error writing to {file_name}: {e}")
        
def createData(concern_label: str, description: str, types: List[str], examples: List[str], number: int = 10) -> List[str]:
    prompt = build_examples_prompt(description, types, examples)
    ret = generate_augmented_examples(prompt,number)
    list_to_txt(concern_label + ".txt", ret)
    print(concern_label + "done")
    return ret


def read_list_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            # Read the file content as a string
            file_content_str = file.read()

            # Safely evaluate the string as a Python literal (list)
            file_contents = ast.literal_eval(file_content_str)

        return file_contents

    except FileNotFoundError:
        print(f"The file '{file_path}' does not exist.")
    except (SyntaxError, ValueError) as e:
        print(f"Error while evaluating the file content: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    
#NC = createData('NC', "there is no concern or problem", ["No concern","Solved problem"], ["I had a problem but I'm good now", "N/A", "none", "no problem", "no issue", "I couldn't find the textbook but I found it now", "I found it hard to understand it but I'm good now", "Linear Algebra didn't make sense to me at first but after grinding I'm good now", "euhrfbourbv ruitovrb outb"],10)
#NC = read_list_from_file("NC.txt")

In [ ]:
listy = read_list_from_file("timeManagement.txt")
listy = flatten(listy)
list_to_txt("timeManagement.txt",listy)

In [ ]:
with open("NC.txt", "r") as file:
    print(file.read())
file_path = 'NC.txt'  # Replace 'your_file.txt' with the actual file path

try:
    with open(file_path, 'r') as file:
        # Read the lines and create a list
        file_contents = file.read().splitlines()

    print("List from the file:")
    print(file_contents)

except FileNotFoundError:
    print(f"The file '{file_path}' does not exist.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
#String generator

def build_examples_prompt(concern_type, concerns_list, examples):
    prompt_builder = f"Give me at least 1000 examples. Augment data for NLP that indicate that there is a {concern_type} concern. Anything that the instructor should look at such as {', '.join(concerns_list)}\nhere are examples\n"
    example_template = "{}\n"


    for example in examples:
        prompt_builder += example_template.format(example)

    return prompt_builder

# Examples
time_management_concerns = ["time management"]
examples_time = [
        "I struggle to keep track of all deadlines",
        "I have a tough time making all the deadlines on time",
        "Too much to keep track of",
        "I don't know how to cram all this knowledge in the limited time I have"
]


examples_other = [
    "I have cancer",
    "I am depressed",
    "I need accomodations for my ADHD",
    "I know people in this class who are cheating",
    "I have a question on how to access the textbook",
    "I need to discuss my 504 plan",
    "This class is giving me anxiety",
    "The tests give me anxiety and make me nervous, I have test anxiety",
    "I need help for my disability",
    "I need accomodations for my wheelechair disability",
    "Would it be okay to come in during office hours to ask for help with things that were covered in pre-requisite courses?",
    "Is there a list of all the vocabulary terms that are useful for this course?",
    "I would like to know if we are going to use any new notations on homework.",
    "Is it ok to work with other people to finish homework?",
    "I broke my hand and need help writing",
    "my mother has covid",
    "I have Covid",
    "I am feeling awful, nothing seems to work and I hate this course",
    "I want to contact the dean of students",
    "How do I set up a meeting with you?",
    "How do I get one on one tutoring?"
    
] 
other_concerns = ["mental health issues", "OSI (office of student integrity)", "Dean of Students", "Disability", "Unstable", "Anxiety", "Accommodation", "ADH", "Disorder", "Depression", "Travel Plans", "504 plan", "any health concern", "any question"]
lm_concerns =["Difficulty Learning material", "Exam Stress", "Reviewing Pre Requisite material for calculus or some other previous class", "Any struggles with learning the material itself", "Finding material difficult"]
examples_lm = [
    "That I forget all my knowledge from calc BC",
    "My only concern about this course is that I will have an issue learning one of the fundamental items early in the course, and as a result of that, will struggle later on when building upon that foundational knowledge. ",
    "The material is really tough and its just not getting to my brain",
    "I'm struggling to learn, I just don't get it",
    "I don't rememeber calculus at all",
    "I don't know how to study for the exams, its so tough to review all that information :(",
    "I found topics related to series really tough last year and I feel like it might be tough to pick up on its content"
    
        
]
time_manage_prompt = build_examples_prompt("time management", time_management_concerns, examples_time)
other_prompt = build_examples_prompt("some other important concern", other_concerns, examples_other)
lm_prompt = build_examples_prompt("learning management concern", lm_concerns, examples_lm)
print("Time Management Prompt:")
print(time_manage_prompt)

print("\nOther Concern Prompt:")
print(other_prompt)

print("\nLearning management Concern Prompt:")
print(lm_prompt)

#NC = createData('NC', "there is no concern or problem", ["No concern","Solved problem"], ["I had a problem but I'm good now", "N/A", "none", "no problem", "no issue", "I couldn't find the textbook but I found it now", "I found it hard to understand it but I'm good now", "Linear Algebra didn't make sense to me at first but after grinding I'm good now", "euhrfbourbv ruitovrb outb"],10)



In [ ]:
# Specify the file path
category 
# Open the file in write mode
with open(file_path, 'w') as file:
    print(x, file=file)

print(f"List has been written to {file_path}")

In [ ]:
responses = x
def flatten(responses, realAug = []):
    n = len(responses)
    augmented = []
    for i in range (0,n):
        augmented.append(responses[i].split('\n'))
   
    for i in range(0, len(augmented)):
        for j in range(0, len(augmented[i])):
            realAug.append(augmented[i][j])
    return realAug

for i in range(0,len(x)):
    holder.append(flatten(no[i]))
len(holder)
